## 2020/4/22の近況報告会を受けて

まずはARモデルで作成した人工時系列を用いて、**きちんとARモデルの係数が推定できているのか**を確認すべき。

$G_{\theta}$と$F_{\phi}$は活性化関数を直線にした一層のニューラルネットで！

In [1]:
import argparse
import os
import numpy as np
from scipy import stats
import math
import sys
import random

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

In [2]:
# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models

ModuleNotFoundError: No module named 'tsModel'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# "output-images"フォルダを作成（既にあるならそれで良し）
os.makedirs("output-images", exist_ok=True)
os.makedirs("parameters", exist_ok=True)

In [ ]:
# 学習時のハイパラの決定（入力を受け付ける）
parser = argparse.ArgumentParser()
parser.add_argument("--p", type=int, default=7, help="ARの次数(generatorへの入力の次元)")
parser.add_argument("--generator_seed", type=int, default=0, help="generatorのパラメータの初期値のシード")
parser.add_argument("--discriminator_seed", type=int, default=0, help="discriminatorのパラメータの初期値のシード")
parser.add_argument("--batch_size", type=int, default=64, help="batchの大きさ")
parser.add_argument("--discriminator_hidden_unit", type=int, default=64, help="discriminatorの隠れ層のニューロンの数")
parser.add_argument("--lr", type=float, default=0.00005, help="学習率")
# opt = parser.parse_args()
opt = parser.parse_args(args=[])

print(opt)

In [ ]:
# gpuが使えるかどうか
cuda = True if torch.cuda.is_available() else False
if cuda:
    print("GPUが使えます。")
    use_gpu = input('GPUを使いますか？ （Yes：1, No：0）  ----> ')
    cuda = bool(int(use_gpu))
else:
    print("GPUは使えません。")
    
if cuda:
    gpu_id = input('使用するGPUの番号を入れてください : ')
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id
device = torch.device('cuda:'+gpu_id if cuda else 'cpu')

In [ ]:
class LinearGenerator(nn.Module):
    def __init__(self, p=7, input_dim=1, is_bias=False): # ネットワークで使う関数を定義する。
        super(LinearGenerator, self).__init__()
        # kernel
        # nn.Conv1dについては、  https://pytorch.org/docs/stable/nn.html#conv1d  を参照
        # self.conv1 = nn.Conv1d(1, n_filter, kernel_size =generator_kernel_size, stride =1, padding=(generator_kernel_size-1)//2, dilation=1, groups=1, bias=True, padding_mode='zeros')
        # self.conv2 = nn.Conv1d(n_filter, n_filter, kernel_size =generator_kernel_size, stride =1, padding=(generator_kernel_size-1)//2, dilation=1, groups=1, bias=True, padding_mode='zeros')
        # self.conv3 = nn.Conv1d(n_filter, n_filter, kernel_size =generator_kernel_size, stride =1, padding=(generator_kernel_size-1)//2, dilation=1, groups=1, bias=True, padding_mode='zeros')
        # self.batchnorm1 = nn.BatchNorm1d(n_filter)
        # self.batchnorm2 = nn.BatchNorm1d(n_filter)
        # self.batchnorm3 = nn.BatchNorm1d(n_filter)
        # 線形変換: y = Wx + b
        self.fc1 = nn.Linear(p+1, 1,bias=is_bias)
            
    def forward(self, x):# ここでネットワークを構成する。入力はx。
        # x = self.conv1(x)
        # x = self.batchnorm1(x)
        # x = F.leaky_relu(x)
        # x = self.conv2(x)
        # x = self.batchnorm2(x)
        # x = F.leaky_relu(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        return x # 出力

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [ ]:
torch.manual_seed(opt.generator_seed)
generator = LinearGenerator(p = opt.p, is_bias=False)

In [ ]:
a= torch.rand([opt.batch_size,1, opt.p+1])
print("input size   :",a.shape)
b = generator(a)
print("output size:",b.shape)

In [ ]:
torch.manual_seed(opt.discriminator_seed)
discriminator = models.Discriminator(q=0, discriminator_hidden_unit=opt.discriminator_hidden_unit)

In [ ]:
a= torch.rand([opt.batch_size, 1])
print("input size   :",a.shape)
b = discriminator(a)
print("output size:",b.shape)

### 人工データの作成

In [ ]:
Data = tsModel.SARIMA(a=[0.3,-0.4,0.3,-0.4,0.3,-0.4,0.3], N=1400, random_seed=0, sigma=2)
Data = torch.tensor(Data)
plt.figure(figsize=(13,8))
plt.plot(Data)
plt.savefig("images/AR7モデルの人工データ.png")
plt.show()

In [ ]:
Data=Data.view(1,-1)

In [ ]:
trainData = Data[:,:1000]
valData = Data[:,1000:1200]
testData = Data[:,1200:]

In [ ]:
trainMatrix = []
for i in range(trainData.shape[1]-(opt.p+1)):
    ans = trainData[:,i:i+opt.p+1].view(1,Data.shape[0],-1)
    trainMatrix.append(ans)
trainMatrix = torch.cat(trainMatrix)
trainMatrix.shape

In [ ]:
valMatrix = []
for i in range(valData.shape[1]-(opt.p+1)):
    ans = valData[:,i:i+opt.p+1].view(1,Data.shape[0],-1)
    valMatrix.append(ans)
valMatrix = torch.cat(valMatrix)
valMatrix.shape

In [ ]:
# Optimizers(パラメータに対して定義される)
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr)

In [ ]:
def gradient_penalty(generated_data, real_data, gp_weight=10):

    batch_size = real_data.size()[0]

    alpha = torch.rand(batch_size, 1)
    alpha = alpha.expand_as(real_data)
    if cuda:
        alpha=alpha.to(device)

    interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
    interpolated = Variable(interpolated, requires_grad=True)
    if cuda:
        interpolated=interpolated.to(device)

    # Calculate probability of interpolated examples
    prob_interpolated = discriminator(interpolated)

    gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                               grad_outputs=torch.ones(prob_interpolated.size()).to(device) if cuda else torch.ones(
                               prob_interpolated.size()),
                               create_graph=True, retain_graph=True)[0]

    gradients = gradients.view(batch_size, -1)# これいらないかも...
    
    # gradients_norm = (gradients.norm(2, dim=1) - 1) ** 2
    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)# 0除算を防ぐ？

    return gp_weight * ((gradients_norm - 1) ** 2).mean()